In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark
!pip install kafka-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
!wget https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.5.0/spark-sql-kafka-0-10_2.12-3.5.0.jar
!wget https://repo1.maven.org/maven2/org/apache/spark/spark-token-provider-kafka-0-10_2.12/3.5.0/spark-token-provider-kafka-0-10_2.12-3.5.0.jar

--2024-12-06 14:36:47--  https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.5.0/spark-sql-kafka-0-10_2.12-3.5.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432335 (422K) [application/java-archive]
Saving to: ‘spark-sql-kafka-0-10_2.12-3.5.0.jar’

spark-sql-kafka-0-1 100%[===================>] 422.20K  --.-KB/s    in 0.06s   

2024-12-06 14:36:47 (6.82 MB/s) - ‘spark-sql-kafka-0-10_2.12-3.5.0.jar’ saved [432335/432335]

--2024-12-06 14:36:47--  https://repo1.maven.org/maven2/org/apache/spark/spark-token-provider-kafka-0-10_2.12/3.5.0/spark-token-provider-kafka-0-10_2.12-3.5.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.


In [ ]:
# Run only once per session
!wget https://downloads.apache.org/kafka/3.7.1/kafka_2.13-3.7.1.tgz
!tar -xvf kafka_2.13-3.7.1.tgz
!rm kafka_2.13-3.7.1.tgz

--2024-12-06 14:39:11--  https://downloads.apache.org/kafka/3.7.1/kafka_2.13-3.7.1.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.208.237, 2a01:4f8:10a:39da::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120235344 (115M) [application/x-gzip]
Saving to: ‘kafka_2.13-3.7.1.tgz’

kafka_2.13-3.7.1.tg 100%[===================>] 114.67M  25.9MB/s    in 5.2s    

2024-12-06 14:39:17 (22.2 MB/s) - ‘kafka_2.13-3.7.1.tgz’ saved [120235344/120235344]

kafka_2.13-3.7.1/
kafka_2.13-3.7.1/LICENSE
kafka_2.13-3.7.1/NOTICE
kafka_2.13-3.7.1/bin/
kafka_2.13-3.7.1/bin/kafka-delete-records.sh
kafka_2.13-3.7.1/bin/trogdor.sh
kafka_2.13-3.7.1/bin/kafka-jmx.sh
kafka_2.13-3.7.1/bin/connect-mirror-maker.sh
kafka_2.13-3.7.1/bin/kafka-console-consumer.sh
kafka_2.13-3.7.1/bin/kafka-consumer-perf-test.sh
kafka_2.13-3.7.1/bin/kafka-log-dirs.sh
kafka_2.13-3.7.1/bin/kafka-met

In [ ]:
import threading

def run_zookeeper():
  !{'kafka_2.13-3.7.1/bin/zookeeper-server-start.sh kafka_2.13-3.7.1/config/zookeeper.properties'}

def run_kafka():
  !{'kafka_2.13-3.7.1/bin/kafka-server-start.sh kafka_2.13-3.7.1/config/server.properties'}

zookeeper_thread = threading.Thread(target=run_zookeeper)
kafka_thread = threading.Thread(target=run_kafka)

zookeeper_thread.start()
kafka_thread.start()


In [ ]:
import subprocess

# Kafka Configuration
KAFKA_BROKER = "localhost:9092"
TOPIC_NAME = "transactions"

def check_and_create_topic():
    # List existing topics
    try:
        result = subprocess.run(
            ['kafka_2.13-3.7.1/bin/kafka-topics.sh', '--list', '--bootstrap-server', KAFKA_BROKER],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True, text=True
        )

        # Check if the topic exists in the list
        if TOPIC_NAME in result.stdout.splitlines():
            print(f"Topic '{TOPIC_NAME}' already exists.")
        else:
            # Create the topic if it does not exist
            print(f"Topic '{TOPIC_NAME}' does not exist. Creating topic...")
            subprocess.run(
                ['kafka_2.13-3.7.1/bin/kafka-topics.sh', '--create', '--topic', TOPIC_NAME,
                 '--bootstrap-server', KAFKA_BROKER, '--partitions', '1', '--replication-factor', '1'],
                check=True
            )
            print(f"Topic '{TOPIC_NAME}' created successfully.")

    except subprocess.CalledProcessError as e:
        print(f"Error checking or creating topic: {e}")

# Run the function to check and create the topic if necessary
check_and_create_topic()


Topic 'transactions' already exists.


In [ ]:
from kafka import KafkaProducer
import csv

# Kafka Configuration
KAFKA_BROKER = "localhost:9092"
TOPIC_NAME = "transactions"

def stream_csv_to_kafka(csv_file_path):
    producer = KafkaProducer(bootstrap_servers=KAFKA_BROKER)

    try:
        with open(csv_file_path, "r") as file:
            reader = csv.DictReader(file)

            # for row in reader:
            #     message = ",".join([f"{key}:{value}" for key, value in row.items()])
            #     producer.send(TOPIC_NAME, value=message.encode("utf-8"))
            #     print(f"Sent: {message}")

            row_count = 0
            for row in reader:
                if row_count >= 10:
                    break  # Stop after sending 10 rows

                message = ",".join([f"{key}:{value}" for key, value in row.items()])
                producer.send(TOPIC_NAME, value=message.encode("utf-8"))
                print(f"Sent: {message}")
                row_count += 1

    except Exception as e:
        print(f"Error: {e}")
    finally:
        producer.close()

# Provide the path to your CSV file
csv_path = "/content/drive/MyDrive/credit-card-fraud-detection-pyspark/dataset/creditcard.csv"
stream_csv_to_kafka(csv_path)


Sent: Time:0,V1:-1.3598071336738,V2:-0.0727811733098497,V3:2.53634673796914,V4:1.37815522427443,V5:-0.338320769942518,V6:0.462387777762292,V7:0.239598554061257,V8:0.0986979012610507,V9:0.363786969611213,V10:0.0907941719789316,V11:-0.551599533260813,V12:-0.617800855762348,V13:-0.991389847235408,V14:-0.311169353699879,V15:1.46817697209427,V16:-0.470400525259478,V17:0.207971241929242,V18:0.0257905801985591,V19:0.403992960255733,V20:0.251412098239705,V21:-0.018306777944153,V22:0.277837575558899,V23:-0.110473910188767,V24:0.0669280749146731,V25:0.128539358273528,V26:-0.189114843888824,V27:0.133558376740387,V28:-0.0210530534538215,Amount:149.62,Class:0
Sent: Time:0,V1:1.19185711131486,V2:0.26615071205963,V3:0.16648011335321,V4:0.448154078460911,V5:0.0600176492822243,V6:-0.0823608088155687,V7:-0.0788029833323113,V8:0.0851016549148104,V9:-0.255425128109186,V10:-0.166974414004614,V11:1.61272666105479,V12:1.06523531137287,V13:0.48909501589608,V14:-0.143772296441519,V15:0.635558093258208,V16:0.46

In [ ]:
from kafka import KafkaConsumer
import time

# Kafka Configuration
KAFKA_BROKER = "localhost:9092"
TOPIC_NAME = "transactions"

def consume_data_from_kafka():
    consumer = KafkaConsumer(
        TOPIC_NAME,
        bootstrap_servers=KAFKA_BROKER,
        auto_offset_reset='earliest',  # Start reading from the earliest message
        group_id=None  # To avoid consumer group issues
    )

    last_message_time = time.time()
    timeout = 3  # Timeout in seconds

    try:
        print("Consuming messages from Kafka topic...")
        while True:
            # Poll the Kafka consumer with a timeout (timeout is in milliseconds)
            records = consumer.poll(timeout_ms=1000)  # Poll every second

            if records:
                # Reset the timeout counter when a message is received
                last_message_time = time.time()
                for topic_partition, messages in records.items():
                    for message in messages:
                        print(f"Received message: {message.value.decode('utf-8')}")
            else:
                # Check if timeout has been exceeded (no new message for the specified timeout)
                if time.time() - last_message_time > timeout:
                    print(f"No new messages for {timeout} seconds, stopping consumer.")
                    break

    except Exception as e:
        print(f"Error: {e}")
    finally:
        consumer.close()

In [ ]:
import time
import logging
from pyspark.sql import SparkSession
from pyspark.sql.streaming import StreamingQueryListener
from pyspark.sql.functions import split, col

# Kafka and Spark Configuration
KAFKA_BROKER = "localhost:9092"
TOPIC_NAME = "transactions"
CHECKPOINT_DIR = "/content/drive/MyDrive/credit-card-fraud-detection-pyspark/checkpoints"
RETRY_ATTEMPTS = 3

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("KafkaConsumerStream")

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("KafkaConsumerFraudDetection") \
    .getOrCreate()

# Define a custom streaming query listener for monitoring
class PerformanceListener(StreamingQueryListener):
    def onQueryProgress(self, event):
        logger.info(f"Batch {event.progress.batchId} processed in {event.progress.batchDurationMs} ms.")
        logger.info(f"Input rows per second: {event.progress.inputRowsPerSecond}")
        logger.info(f"Processed rows per second: {event.progress.processedRowsPerSecond}")

    def onQueryStarted(self, event):
        logger.info(f"Query started: {event.id}")

    def onQueryTerminated(self, event):
        logger.info(f"Query terminated: {event.id}")

spark.streams.addListener(PerformanceListener())


def process_streaming_data():

    # Read data from Kafka
    # transaction_data = spark.readStream \
    # .format("kafka") \
    # .option("kafka.bootstrap.servers", KAFKA_BROKER) \
    # .option("subscribe", TOPIC_NAME) \
    # .option("startingOffsets", "earliest") \
    # .option("checkpointLocation", CHECKPOINT_DIR) \
    # .load()

    # Read data from Kafka
    transaction_data = consume_data_from_kafka()

    # Preprocess, as required
    # This is where data preprocessing can be added if necessary

    # Predict, using the model
    # predictions = model.transform(transaction_data)

    # Write predictions to console
    # predictions.writeStream \
    #     .format("console") \
    #     .outputMode("append") \
    #     .option("checkpointLocation", CHECKPOINT_DIR) \
    #     .start() \
    #     .awaitTermination()


def start_streaming():
    # Stream Processing Logic with Error Handling and Retries
    for attempt in range(RETRY_ATTEMPTS):
        try:
            logger.info("Starting streaming process...")
            process_streaming_data()  # Process the data
            break  # Exit loop if successful
        except Exception as e:
            logger.error(f"Attempt {attempt + 1} failed: {e}")
            time.sleep(5)  # Retry delay
    else:
        logger.critical("All retry attempts failed. Exiting application.")


# Start streaming with retry logic
start_streaming()


Consuming messages from Kafka topic...
Received message: Time:0,V1:-1.3598071336738,V2:-0.0727811733098497,V3:2.53634673796914,V4:1.37815522427443,V5:-0.338320769942518,V6:0.462387777762292,V7:0.239598554061257,V8:0.0986979012610507,V9:0.363786969611213,V10:0.0907941719789316,V11:-0.551599533260813,V12:-0.617800855762348,V13:-0.991389847235408,V14:-0.311169353699879,V15:1.46817697209427,V16:-0.470400525259478,V17:0.207971241929242,V18:0.0257905801985591,V19:0.403992960255733,V20:0.251412098239705,V21:-0.018306777944153,V22:0.277837575558899,V23:-0.110473910188767,V24:0.0669280749146731,V25:0.128539358273528,V26:-0.189114843888824,V27:0.133558376740387,V28:-0.0210530534538215,Amount:149.62,Class:0
Received message: Time:0,V1:1.19185711131486,V2:0.26615071205963,V3:0.16648011335321,V4:0.448154078460911,V5:0.0600176492822243,V6:-0.0823608088155687,V7:-0.0788029833323113,V8:0.0851016549148104,V9:-0.255425128109186,V10:-0.166974414004614,V11:1.61272666105479,V12:1.06523531137287,V13:0.48909

ERROR:kafka.consumer.fetcher:Fetch to node 0 failed: Cancelled: <BrokerConnection node_id=0 host=4ac4ffb6490d:9092 <connected> [IPv4 ('172.28.0.12', 9092)]>


No new messages for 3 seconds, stopping consumer.
